In [ ]:
#per Connection
import socket
HOST = "127.0.0.1"
PORT = 4505
PACKET_SIZE = 4096

#per prometheus
import requests
from datetime import datetime
PROMETHEUS_SERVER = "http://192.168.2.3:9090/"

#varie
from IPython.display import clear_output


with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as MySocket:
    MySocket.bind((HOST,PORT))
    MySocket.listen()

    while True:
        RemoteConnection, RemoteAddress = MySocket.accept()
        clear_output(wait=True)
        
        with RemoteConnection:
            print(f"Connection received from {RemoteAddress}")
            while True: 
                ReceivedData = RemoteConnection.recv(PACKET_SIZE)

                # check if is there some data
                if not ReceivedData:
                    break

                # print the received data
                print(ReceivedData)

                #request data from Prom
                QueryDataT = requests.get(PROMETHEUS_SERVER + "/api/v1/query",params={'query': "generic_temp_c"})
                QueryDataJsonT = QueryDataT.json()

                TValue = QueryDataJsonT["data"]["result"][0]["value"][1]
                TTime = datetime.utcfromtimestamp(int(QueryDataJsonT["data"]["result"][0]["value"][0])).strftime('%Y-%m-%d %H:%M:%S')

                StringToSend = str(TValue) + "&" + TTime

                # response
                RemoteConnection.sendall(bytes(StringToSend,"utf-8"))

                # exit from loop is when client terminate